In [ ]:
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType
from datetime import datetime

In [ ]:
sqlContext = SQLContext(sc)
df = sqlContext.read.format("com.databricks.spark.csv").options(header="true", inferschema="true", dateFormat="%m/%d/%y %H:%M").load("../data/total_fixed.csv")

In [ ]:
func =  udf (lambda x: datetime.strptime(x, '%m/%d/%y %H:%M'), DateType())

In [ ]:
df = df.withColumn('Changed', func(col('Changed')))
df = df.withColumn('Opened', func(col('Opened')))

In [ ]:
df.show()

In [ ]:
data = pd.read_csv('../data/total_fixed.csv')
df = sqlContext.createDataFrame(data)

In [ ]:
print (data)

Multi processing Pandas Data Frame 

In [ ]:
from multiprocessing import Pool
num_partitions = 10 #number of partitions to split dataframe
num_cores = 4 #number of cores on your machine

In [ ]:
def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [ ]:
def multiply_columns(data):
    data['TMP1'] = data['Opened'].apply(lambda x: tmp1(x, 7))
    return data

In [ ]:
data2 = parallelize_dataframe(data, multiply_columns)

Generating Temporal Features (Slow Version)

In [ ]:
# Generating Temporal Features (slow version)
def tmp1(x, n):
    return sum(data['Opened'] < x) - sum (data['Opened'] < (x - timedelta(days=n)))

def tmp1_np(x, n):
    return sum(np_opened < x) - sum (np_opened < (x - np.timedelta64(n, 'D')))


def tmp2(x, n):
    subdata = data[data['Severity'] == x['Severity']]
    return sum(subdata['Opened'] < x['Opened']) - sum (subdata['Opened'] < (x['Opened'] - timedelta(days=n)))

def subdatagen(x, severity):
    if severity_levels[x['Severity']] >= severity_levels[severity]:
        return True
    else:
        return False

def tmp3(i, n):
    condn = data.apply(lambda x: subdatagen(x, i['Severity']), axis=1)
    subdata = data[condn]
    return sum(subdata['Opened'] < x['Opened']) - sum (subdata['Opened'] < (x['Opened'] - timedelta(days=n)))

In [ ]:
start = time()
tmp1(data['Opened'].iloc[0], 7)
end = time() - start
print "Time taken = ", end